In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
sns.set_theme(rc={'figure.figsize':(10,6)})
sns.set_style("whitegrid")

In [10]:
values = [
    'mymodel2_0_woN_1500',
    'mymodel2_1_wN_1500',
    'mymodel2_2_woN_1500',
    'mymodel2_2_wN_1500',
]

In [11]:
dfs = []
for value in values:
    df = pd.read_csv(f'../data/validation_normalization/metrics_{value}.csv')
    elos = []
    for elo in list(df['elo']):
        elos.append((elo//100)*100)
    
    df['fixed_elo'] = elos
    new_df = df[['accuracy', 'f1_weighted', 'elo', 'fixed_elo']].groupby('fixed_elo').mean()
    new_df = new_df.reset_index()
    new_df['model'] = df.loc[0]['model']
    new_df['normalization'] = "False" if df.loc[0]['model'] == "mymodel2_2_woN_1500" or df.loc[0]['model'] == "mymodel2_0_woN_1500" else "True"
    new_df = new_df.loc[(new_df['fixed_elo'] >= 1000.0)]
    new_df = new_df.loc[(new_df['fixed_elo'] <= 2000.0)]
    dfs.append(new_df)
results = pd.concat(dfs)

In [13]:
models = [
    'mymodel2_2_woN_1500',
    'mymodel2_2_wN_1500',
]
plot = sns.lineplot(data=results[results['model'].isin(models)], x='fixed_elo', y='accuracy', hue='normalization', palette=sns.color_palette("husl", 2))
sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))
plot.set(xlabel='elo', xticks=list(range(1000, 2100, 100)))
fig = plot.get_figure()
fig.savefig(f'../docs/plots/plot_woN_vs_wN.pdf', bbox_inches='tight')
plt.close()